In [1]:
import sys
COMP_NAME = "icecube-neutrinos-in-deep-ice"
sys.path.append(f"/home/anjum/kaggle/{COMP_NAME}/")

import pandas as pd
import numpy as np
import torch

from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

from graphnet.models.gnn import DynEdge
from graphnet.models.graph_builders import KNNGraphBuilder

from src.config import INPUT_PATH, OUTPUT_PATH

graphnet: INFO     2023-01-22 18:58:54 - get_logger - Writing log to logs/graphnet_20230122-185854.log


In [42]:
_dtype = {
    "batch_id": "int16",
    "event_id": "int64",
    "first_pulse_index": "int32",
    "last_pulse_index": "int32",
    "azimuth": "float32",
    "zenith": "float32",
}

meta = pd.read_parquet(INPUT_PATH / "train_meta.parquet").astype(_dtype)
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131953924 entries, 0 to 131953923
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   batch_id           int16  
 1   event_id           int64  
 2   first_pulse_index  int32  
 3   last_pulse_index   int32  
 4   azimuth            float32
 5   zenith             float32
dtypes: float32(2), int16(1), int32(2), int64(1)
memory usage: 3.2 GB


In [43]:
meta.describe()

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
count,1.319539e+08,1.319539e+08,1.319539e+08,1.319539e+08,1.319539e+08,1.319539e+08
mean,3.303849e+02,1.073797e+09,1.643336e+07,1.643352e+07,3.144117e+00,1.534421e+00
std,1.904591e+02,6.199853e+08,9.505803e+06,9.505803e+06,1.813510e+00,6.901115e-01
min,1.000000e+00,2.400000e+01,0.000000e+00,2.700000e+01,6.436839e-08,8.631674e-05
25%,1.650000e+02,5.368332e+08,8.202315e+06,8.202470e+06,1.575369e+00,1.005033e+00
50%,3.300000e+02,1.073825e+09,1.641943e+07,1.641959e+07,3.141370e+00,1.526995e+00
75%,4.950000e+02,1.610750e+09,2.465248e+07,2.465266e+07,4.717619e+00,2.054609e+00
max,6.600000e+02,2.147484e+09,3.553534e+07,3.553541e+07,6.283185e+00,3.141562e+00


In [28]:
batch_id = 1
event_id = 24

sensors = pd.read_csv(INPUT_PATH / "sensor_geometry_v2.csv", index_col="sensor_id")
meta = pd.read_parquet(INPUT_PATH / "train_meta.parquet").query(f"batch_id == {batch_id}")
batch = pd.read_parquet(INPUT_PATH / "train" / f"batch_{batch_id}.parquet")

In [35]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   batch_id           200000 non-null  int64  
 1   event_id           200000 non-null  Int64  
 2   first_pulse_index  200000 non-null  int64  
 3   last_pulse_index   200000 non-null  int64  
 4   azimuth            200000 non-null  float64
 5   zenith             200000 non-null  float64
dtypes: Int64(1), float64(2), int64(3)
memory usage: 10.9 MB


In [29]:
sensors.head()

,x,y,z,string,qe
sensor_id,,,,,
0,-256.14,-521.08,496.03,0,1.0
1,-256.14,-521.08,479.01,0,1.0
2,-256.14,-521.08,461.99,0,1.0
3,-256.14,-521.08,444.97,0,1.0
4,-256.14,-521.08,427.95,0,1.0


In [30]:
meta.head()

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117


In [31]:
batch.head()

,sensor_id,time,charge,auxiliary
event_id,,,,
24,3918,5928,1.325,True
24,4157,6115,1.175,True
24,3520,6492,0.925,True
24,5041,6665,0.225,True
24,2948,8054,1.575,True


In [32]:
sensors.head()

,x,y,z,string,qe
sensor_id,,,,,
0,-256.14,-521.08,496.03,0,1.0
1,-256.14,-521.08,479.01,0,1.0
2,-256.14,-521.08,461.99,0,1.0
3,-256.14,-521.08,444.97,0,1.0
4,-256.14,-521.08,427.95,0,1.0


In [34]:
%%timeit
event = pd.merge(batch.loc[event_id], sensors, on="sensor_id")

# event = batch.loc[event_id].set_index("sensor_id", drop=True).join(sensors)
event

1.33 ms ± 3.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [44]:
def preprocessing(event):
    event["x"] /= 500
    event["y"] /= 500
    event["z"] /= 500
    event["time"] = (event["time"] - 1.0e04) / 3.0e4
    event["charge"] = np.log10(event["charge"]) / 3.0
    event["auxiliary"] = event["auxiliary"].astype(int)
    
    return event[["x", "y", "z", "time", "charge", "qe", "auxiliary"]]


event = preprocessing(event)
event

,x,y,z,time,charge,qe,auxiliary
0,0.60682,0.67128,0.41316,-0.135733,0.040739,1.0,1
1,-0.29090,0.74848,0.42546,-0.129500,0.023346,1.0,1
2,1.01054,0.51576,-0.34920,-0.116933,-0.011286,1.0,1
3,1.01054,0.51576,-0.34920,0.150767,0.040739,1.0,1
4,-0.01936,-0.15900,0.36200,-0.111167,-0.215939,1.0,1
...,...,...,...,...,...,...,...
56,0.76470,0.47780,-0.09224,0.260400,-0.046554,1.0,1
57,-0.51228,-1.04216,-0.64194,0.268433,-0.003665,1.0,1
58,0.02374,0.35838,0.08556,0.269833,0.017051,1.0,1
59,0.02374,0.35838,0.08556,0.270067,0.051272,1.0,1


In [45]:
event.describe()

,x,y,z,time,charge,qe,auxiliary
count,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000
mean,0.023824,-0.058633,0.052818,0.092286,-0.020030,1.034426,0.786885
std,0.610632,0.520048,0.502226,0.113998,0.078332,0.105096,0.412907
min,-1.053260,-1.042160,-0.876160,-0.135733,-0.252321,1.000000,0.000000
25%,-0.469900,-0.535040,-0.410940,0.021400,-0.046554,1.000000,1.000000
50%,0.023740,-0.031200,0.204360,0.086900,-0.011286,1.000000,1.000000
75%,0.317880,0.358380,0.424260,0.187800,0.029379,1.000000,1.000000
max,1.152740,0.980440,0.999020,0.301033,0.199779,1.350000,1.000000


In [46]:
x = torch.tensor(event.values, dtype=torch.float32)
x.shape, x.dtype

(torch.Size([61, 7]), torch.float32)

In [56]:
target = meta.query(f"batch_id == {batch_id} & event_id == {event_id}")
y = torch.tensor(target[["azimuth", "zenith"]].values, dtype=torch.float32)
y

tensor([[5.0296, 2.0875]])

In [57]:
data = Data(x=x, y=y)
data.n_pulses = torch.tensor(x.shape[0], dtype=torch.int32)
data

Data(x=[61, 7], y=[1, 2], n_pulses=61)

In [48]:
builder = KNNGraphBuilder(nb_nearest_neighbours=8)

data_out = builder(data)
data_out

Data(x=[61, 7], n_pulses=61, edge_index=[2, 488])

In [49]:
# In final version use a PTGeo dataset & dataloader
batch = Batch.from_data_list([data_out, data_out])
batch

DataBatch(x=[122, 7], n_pulses=[2], edge_index=[2, 976], batch=[122], ptr=[3])

In [50]:
model = DynEdge(nb_inputs=7, global_pooling_schemes=["min", "max", "mean", "sum"])

In [51]:
out = model(batch)
out.shape

torch.Size([2, 128])

In [52]:
torch.save(data, "test.pt")

In [53]:
test = torch.load("test.pt")
test

Data(x=[61, 7], n_pulses=61, edge_index=[2, 488])